In [1]:
using Statistics

In [2]:
abstract type ModelType end

abstract type Categorical <: ModelType end

abstract type Classifier <: Categorical end

abstract type Continuous <: ModelType end

In [3]:
abstract type AbstractModel end

mutable struct Model{T <: Any} <: AbstractModel
    data::Dict{String, Any}
    Model{T}() where {T <: ModelType} = new{T}(Dict{String, Any}())
end

In [4]:
function accuracy(model::AbstractModel, testx::Vector{<:Any}, testy::Vector{<:Any})
    yhat::Vector{<:Any} = predict(model, testx)
    bitmask::Vector{Bool} = [begin 
        if x == y 
           true
        else 
           false
        end
    end for (x, y) in zip(yhat, testy)]
    (length(bitmask) / length(findall(x -> x, bitmask)))::Real
end

accuracy (generic function with 1 method)

In [16]:
function accuracy(model::Model{<:Continuous}, testx::Vector{<:Any}, testy::Vector{<:Any})
    yhat::Vector{<:Any} = predict(model, testx)
    (cor(yhat, testy) ^ 2)::Number
end

accuracy (generic function with 2 methods)

In [17]:
abstract type MajorityClass <: Categorical end

abstract type MeanBaseline <: Continuous end

In [18]:
MajorityClass() = Model{MajorityClass}()
MeanBaseline() = Model{MeanBaseline}()

MeanBaseline

In [19]:
function fit!(model::Model{MeanBaseline}, trainx::Vector{<:Any}, trainy::Vector{<:Any})
    push!(model.data, "mu" => Float64(sum(trainy) / length(trainy)))
end

fit! (generic function with 1 method)

In [20]:
predict(model::Model{MeanBaseline}, testx) = begin
    if ~("mu" in keys(model.data))
        throw("this model has not been fitted to data yet. Use `fit!` first.")
    end
    mean::Real = model.data["mu"]
    [mean for x in testx]::Vector{<:Any}
end

predict (generic function with 1 method)

In [21]:
trainx = [5, 5, 6, 2, 3, 7, 5]
trainy = [5, 8, 4, 3, 3, 4, 6]
testx = [5, 6, 3, 4]
testy = [5, 7, 4, 4]

4-element Vector{Int64}:
 5
 7
 4
 4

In [22]:
mymodel = MeanBaseline()

Model{MeanBaseline}(Dict{String, Any}())

In [23]:
fit!(mymodel, trainx, trainy)

Dict{String, Any} with 1 entry:
  "mu" => 4.71429

In [24]:
predict(mymodel, testx)

4-element Vector{Float64}:
 4.714285714285714
 4.714285714285714
 4.714285714285714
 4.714285714285714

In [25]:
accuracy(mymodel, testx, testy)

[4.714285714285714, 4.714285714285714, 4.714285714285714, 4.714285714285714]


NaN